In [1]:
import numpy as np
import pandas as pd


In [11]:
fake_df=pd.read_csv('Fake.csv')
true_df=pd.read_csv('True.csv')

In [12]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [13]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [14]:
fake_df.shape

(23481, 4)

In [15]:
true_df.shape

(21417, 4)

In [16]:
## dropping unwanted columns 

In [17]:
fake_df.drop(columns=['title','subject','date'],inplace=True)

In [18]:
true_df.drop(columns=['title','subject','date'],inplace=True)

In [19]:
fake_df.columns

Index(['text'], dtype='str')

In [20]:
true_df.columns

Index(['text'], dtype='str')

In [21]:
## checking for null values

In [23]:
fake_df.isna().sum()
true_df.isna().sum()

text    0
dtype: int64

In [24]:
fake_df['label']=1
true_df['label']=0

In [32]:
fake_df.head()
# true_df.head()

,text,label
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1
2,"On Friday, it was revealed that former Milwauk...",1
3,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis used his annual Christmas Day mes...,1


In [28]:
df=pd.concat([fake_df,true_df],axis=0)

In [29]:
df.shape

(44898, 2)

In [30]:
# Text preprocessing

In [31]:
import nltk

In [33]:
import re

def preprocess_text(text):
    # convert to string (safety)
    text = str(text)
    
    # lowercase
    text = text.lower()
    
    # remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # remove numbers and special characters
    text = re.sub(r'[^a-z\s]', ' ', text)
    
    # remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text


In [34]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Saniya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Saniya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Saniya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Saniya\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Saniya\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [35]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


In [36]:
from nltk.corpus import wordnet

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


In [40]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Negation words to KEEP
negation_words = {'no', 'not', 'nor', 'never', 'none', 'nobody', 'nothing', 'neither', 'nowhere'}

# Remove negation words from stopwords
stop_words = stop_words - negation_words

print(sorted(list(negation_words & set(stopwords.words('english')))))


['no', 'nor', 'not']


In [41]:
def advanced_preprocess(text):
    text = str(text).lower()
    
    # remove urls
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # remove special characters & numbers
    text = re.sub(r'[^a-z\s]', ' ', text)
    
    # tokenize
    tokens = word_tokenize(text)
    
    # remove stopwords but keep negations
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    
    # POS tagging
    pos_tags = nltk.pos_tag(tokens)
    
    # Lemmatization with POS
    lemmatized_tokens = [
        lemmatizer.lemmatize(word, get_wordnet_pos(pos))
        for word, pos in pos_tags
    ]
    
    return " ".join(lemmatized_tokens)


In [42]:
df['processed_text'] = df['text'].apply(advanced_preprocess)


In [43]:
df[['text', 'processed_text']].head(3)


,text,processed_text
0,Donald Trump just couldn t wish all Americans ...,donald trump wish american happy new year leav...
1,House Intelligence Committee Chairman Devin Nu...,house intelligence committee chairman devin nu...
2,"On Friday, it was revealed that former Milwauk...",friday reveal former milwaukee sheriff david c...


In [44]:
df['original_len'] = df['text'].apply(lambda x: len(str(x).split()))
df['processed_len'] = df['processed_text'].apply(lambda x: len(x.split()))

df[['original_len', 'processed_len']].describe()


,original_len,processed_len
count,44898.000000,44898.000000
mean,405.282284,231.288454
std,351.265595,201.897346
min,0.000000,0.000000
25%,203.000000,117.000000
50%,362.000000,205.000000
75%,513.000000,290.000000
max,8135.000000,4910.000000


In [45]:
# Feature Engineering

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=5,
    max_df=0.9,
    sublinear_tf=True
)


In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

lr_model = Pipeline([
    ('tfidf', tfidf),
    ('clf', LogisticRegression(
        max_iter=1000,
        class_weight='balanced',
        solver='liblinear'
    ))
])


In [50]:
X = df['processed_text']
y = df['label']


In [51]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [52]:
lr_model.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('tfidf', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None


In [53]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, classification_report, confusion_matrix
)

y_pred = lr_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.9908685968819599
Precision: 0.994639794168096
Recall: 0.987862010221465
F1 Score: 0.9912393162393163

Classification Report:

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4284
           1       0.99      0.99      0.99      4696

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980


Confusion Matrix:

[[4259   25]
 [  57 4639]]


In [54]:
train_acc = lr_model.score(X_train, y_train)
test_acc = lr_model.score(X_test, y_test)

print("Training Accuracy:", train_acc)
print("Testing Accuracy:", test_acc)

Training Accuracy: 0.9949329027228687
Testing Accuracy: 0.9908685968819599


In [55]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(
    lr_model,
    X,
    y,
    cv=cv,
    scoring='f1'
)

print("CV F1 Scores:", cv_scores)
print("Mean CV F1:", cv_scores.mean())

CV F1 Scores: [0.98793122 0.98974359 0.99112394 0.99039693 0.99241048]
Mean CV F1: 0.990321231006282


In [56]:
from sklearn.svm import LinearSVC

svm_model = Pipeline([
    ('tfidf', tfidf),
    ('clf', LinearSVC(class_weight='balanced'))
])

svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

print(classification_report(y_test, y_pred_svm))
print(confusion_matrix(y_test, y_pred_svm))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4284
           1       1.00      1.00      1.00      4696

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980

[[4278    6]
 [  16 4680]]


In [58]:
train_acc = svm_model.score(X_train, y_train)
test_acc = svm_model.score(X_test, y_test)

print("Training Accuracy:", train_acc)
print("Testing Accuracy:", test_acc)

Training Accuracy: 0.9999721588061696
Testing Accuracy: 0.9975501113585746


In [57]:
import pandas as pd

error_df = pd.DataFrame({
    'text': X_test,
    'true': y_test,
    'pred': y_pred
})

error_df[error_df['true'] != error_df['pred']].head(10)


,text,true,pred
18458,ford export next generation focus compact car ...,1,0
6396,new york reuters drama start early donald trum...,0,1
17083,many day remain community organizer chief term...,1,0
1700,los angeles reuters emmy award show peppered p...,0,1
17475,accord sun george bush successor barack obama ...,1,0
9967,wfb north korea first time week reveal plan us...,1,0
20368,anti establishment tide roll across europe rea...,1,0
17565,accord cnn prime minister theresa may deliver ...,1,0
11030,new york reuters contender white house use soc...,0,1
1892,reuters trump administration draw vigorous res...,0,1


In [ ]:
# This dataset is stylistically separable. Fake news uses emotional and opinionated language, while real news follows formal journalistic tone. Linear models like SVM with TF-IDF n-grams perform exceptionally well here, which is also supported by cross-validation.”

In [59]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(
    svm_model,
    X,
    y,
    cv=5,
    scoring='f1'
)

print("CV F1 Scores:", cv_scores)
print("Mean CV F1:", cv_scores.mean())



CV F1 Scores: [0.99702507 0.99459345 0.99361702 0.99292453 0.99208048]
Mean CV F1: 0.9940481103870857


In [61]:
import numpy as np

vectorizer = svm_model.named_steps['tfidf']
classifier = svm_model.named_steps['clf']

feature_names = vectorizer.get_feature_names_out()
coef = classifier.coef_[0]


top_fake = np.argsort(coef)[-20:]
top_real = np.argsort(coef)[:20]

print("Top Fake News Indicators:")
print(feature_names[top_fake])

print("\nTop Real News Indicators:")
print(feature_names[top_real])



Top Fake News Indicators:
['via reuters' 'century wire' 'entire' 'feature' 'getty image' 'com'
 'image via' 'getty' 'rep' 'wire' 'president obama' 'america' 'sen'
 'feature image' 'obama' 'gop' 'image' 'president trump' 'read' 'via']

Top Real News Indicators:
['reuters' 'say' 'washington reuters' 'president donald'
 'reuters president' 'wednesday' 'tuesday' 'washington' 'thursday'
 'york reuters' 'nov' 'monday' 'friday' 'minister' 'say statement'
 'president barack' 'republican' 'london reuters' 'representative'
 'london']
